# Aula 2: Pipeline, Validação e Treinamento (com validação pandas)
Validação de dados, treino, avaliação e rastreamento de experimentos.

In [26]:
%pip install mlflow

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib as joblib
import mlflow as mlflow
import os as os

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Passo 1: Carregue os dados
caminho = "../data/tweets_limpo.csv"
if not os.path.exists(caminho): 
    raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")

df = pd.read_csv(caminho)

In [28]:
# Passo 2: Validação simples com pandas (substituindo Great Expectations)
assert df["text"].notnull().all(), "Coluna 'text' possui valores nulos." 
assert df["label"].notnull().all(), "Coluna 'label' possui valores nulos."
assert df["text"].apply(lambda x: isinstance(x, str)).all(), "Coluna 'text' não é do tipo string."
assert df["label"].isin(["positivo", "negativo","neutro"]).all(), "Coluna 'label' possui valores inesperados."
print("Validação dos dados concluída com sucesso!")

Validação dos dados concluída com sucesso!


In [29]:
# Passo 3: Separe treino e teste
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

In [30]:
# Passo 4: Crie e treine o pipeline
pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("clf", MultinomialNB())
])
pipeline.fit(X_train, y_train)

,steps,"[('vect', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [31]:
# Passo 5: Avalie o modelo
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, labels=["positivo", "neutro", "negativo"], digits=2, zero_division=0))

              precision    recall  f1-score   support

    positivo       1.00      1.00      1.00         3
      neutro       0.50      0.33      0.40         3
    negativo       0.50      0.67      0.57         3

    accuracy                           0.67         9
   macro avg       0.67      0.67      0.66         9
weighted avg       0.67      0.67      0.66         9



Salvar o modelo e vetor

In [32]:
joblib.dump(pipeline.named_steps["clf"], "../model.joblib")
joblib.dump(pipeline.named_steps["vect"], "../vectorizer.joblib")

['../vectorizer.joblib']

Rastrear os experimentos com MLFLOW

In [33]:
from mlflow.models import infer_signature

input_example = pd.DataFrame({"text": ["Esse produto é ótimo!"]})

mlflow.sklearn.log_model(
    sk_model=pipeline, 
    name="modelo_sentimento", 
    input_example=input_example
)